In [1]:
import gc
import time
from logzero import logger
import numpy as np
import pandas as pd

In [2]:
ls -lah ../data/

total 24G
drwxrwxr-x 5 ubuntu ubuntu 4.0K May  5 15:34 ./
drwxrwxr-x 8 ubuntu ubuntu 4.0K Apr 30 17:30 ../
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 16:52 .ipynb_checkpoints/
-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 13:04 count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 count_user_by_channel_app.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 29 14:21 kenkoooos/
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 merge.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 17:13 raw/
-rw-rw-r-- 1 ubuntu ubuntu 1.4G Apr 28 17:20 raw.zip
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May

In [ ]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=8)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs


In [ ]:
merge = merge[merge.is_test == False]
gc.collect()

In [ ]:
print(merge.shape)
merge.dtypes

### preparation

In [6]:
%%time
channel_grp = merge.groupby('app', as_index=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 246 µs


In [7]:
del merge
gc.collect()

0

### simple count

In [8]:
%%time
count_by_app = (channel_grp.ip.count().rename(columns={'ip': 'count_by_app'}))

CPU times: user 2.37 s, sys: 1.21 s, total: 3.58 s
Wall time: 3.58 s


In [9]:
print(count_by_app.shape)
count_by_app.head()

(674, 2)


,app,count_by_app
0,0,2609
1,1,3795936
2,2,14878211
3,3,23614633
4,4,89155


In [10]:
%%time
attrib_by_app = (channel_grp.is_attributed.sum().rename(columns={'is_attributed': 'attrib_by_app'})).astype('uint32')

CPU times: user 1.01 s, sys: 584 ms, total: 1.59 s
Wall time: 1.59 s


In [11]:
attrib_by_app.head()

,app,attrib_by_app
0,0,791
1,1,617
2,2,3504
3,3,6900
4,4,3


In [12]:
agg_by_app = pd.concat([count_by_app, 
                        attrib_by_app['attrib_by_app']], axis=1)

In [13]:
agg_by_app.to_feather('../data/agg_by_app.feather')

In [14]:
del agg_by_app
gc.collect()

14

In [15]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 12:36 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 12:31 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### unique count

In [16]:
%%time
uqcount_by_app = pd.concat([
    channel_grp.channel.nunique().to_frame('uqcount_channel_by_app')
    , channel_grp.ip.nunique().to_frame('uqcount_ip_by_app')
    , channel_grp.device.nunique().to_frame('uqcount_device_by_app')
    , channel_grp.os.nunique().to_frame('uqcount_os_by_app')
    , channel_grp.ip_os_device_dow_hour.nunique().to_frame('uqcount_user_by_app')
], axis=1)

CPU times: user 3min 28s, sys: 6.09 s, total: 3min 34s
Wall time: 3min 39s


In [17]:
uqcount_by_app.head()

,uqcount_channel_by_app,uqcount_ip_by_app,uqcount_device_by_app,uqcount_os_by_app,uqcount_user_by_app
0,1,1139,227,14,1417
1,32,113943,6,209,2479017
2,27,135317,6,351,5288033
3,47,155393,7,280,7219771
4,1,10502,6,104,38886


In [18]:
uqcount_by_app.to_feather('../data/uqcount_by_app.feather')

In [19]:
del uqcount_by_app
gc.collect()

69

In [20]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 12:36 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### cross tabulation(channel)

In [ ]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=8)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.1 µs


In [4]:
merge = merge[['channel', 'app', 'ip_os_device_dow_hour']][merge.is_test == False]
merge['counter'] = 1
gc.collect()

0

In [5]:
print(merge.shape)
merge.head()

(126923238, 4)


,channel,app,ip_os_device_dow_hour,counter
0,153,23,765080350001103,1
1,134,18,765080350001103,1
2,315,15,765080350001103,1
3,469,11,765080350001103,1
4,379,3,765080350001103,1


In [24]:
merge.drop_duplicates(inplace = True)

In [25]:
print(merge.shape)
merge.head()

(83747805, 4)


,channel,app,ip_os_device_dow_hour,counter
0,153,23,765080350001103,1
1,134,18,765080350001103,1
2,315,15,765080350001103,1
3,469,11,765080350001103,1
4,379,3,765080350001103,1


In [26]:
%%time
count_user_by_channel_app = merge.pivot_table(values  = 'counter',
                                              index   = 'app',
                                              columns = 'channel', 
                                              aggfunc = sum,
                                              fill_value = 0)

CPU times: user 4.62 s, sys: 2.46 s, total: 7.08 s
Wall time: 7.08 s


In [27]:
chlabel = ['ch_{0:03d}'.format(i) for i in list(set(merge.channel))]
count_user_by_channel_app.columns = chlabel
count_user_by_channel_app.reset_index(inplace=True)

In [28]:
count_user_by_channel_app.head()

,app,ch_000,ch_003,ch_004,ch_005,ch_013,ch_014,ch_015,ch_017,ch_018,...,ch_484,ch_486,ch_487,ch_488,ch_489,ch_490,ch_496,ch_497,ch_498,ch_500
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,4,0,0,36209,0,0,147786,5141,...,0,0,0,0,0,0,0,1,0,0
2,2,0,0,0,0,0,0,0,0,26665,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,3942,745567,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
count_user_by_channel_app.tail()

,app,ch_000,ch_003,ch_004,ch_005,ch_013,ch_014,ch_015,ch_017,ch_018,...,ch_484,ch_486,ch_487,ch_488,ch_489,ch_490,ch_496,ch_497,ch_498,ch_500
669,764,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
670,765,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
671,766,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
672,767,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
673,768,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
count_user_by_channel_app.to_feather('../data/count_user_by_app_channel.feather')

In [31]:
del count_user_by_channel_app
gc.collect()

1829

In [32]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 13:04 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### cross tabulation (apps clicked by the same user)

In [3]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=64)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.7 µs


In [4]:
merge = merge[['app', 'ip_os_device_dow_hour']][merge.is_test == False]
merge['counter'] = 1

In [5]:
merge.drop_duplicates(inplace=True)

In [6]:
print(merge.shape)
merge.head()

(57046774, 3)


,app,ip_os_device_dow_hour,counter
0,23,765080350001103,1
1,18,765080350001103,1
2,15,765080350001103,1
3,11,765080350001103,1
4,3,765080350001103,1


In [7]:
gc.collect()

0

In [8]:
merge['counter'] = merge['counter'].astype('bool')

In [9]:
%%time
d = merge.set_index(['ip_os_device_dow_hour', 'app']).unstack(level=-1)

CPU times: user 17min 22s, sys: 5min 52s, total: 23min 15s
Wall time: 23min 7s


In [10]:
d.head()

counter                                                  \
app                       0     1     2     3     4     5     6     7     8     
ip_os_device_dow_hour                                                           
10020001303              None  None  True  None  None  None  None  None  None   
10020001314              None  True  None  None  None  None  None  None  None   
10040001312              None  None  None  None  None  None  None  None  None   
10040001314              None  None  None  None  None  True  None  None  None   
10090001303              None  True  True  None  None  None  None  None  True   

                             ...                                             \
app                     9    ...    758   759   760   761   763   764   765   
ip_os_device_dow_hour        ...                                              
10020001303            None  ...   None  None  None  None  None  None  None   
10020001314            None  ...   None  None  None  None  None  None  None   
10040001312            None  ...   None  None  None  None  None  None  None   
10040001314            None  ...   None  None  None  None  None  None  None   
10090001303            True  ...   None  None  None  None  None  None  None   

                                         
app                     766   767   768  
ip_os_device_dow_hour                    
10020001303            None  None  None  
10020001314            None  None  None  
10040001312            None  None  None  
10040001314            None  None  None  
10090001303            None  None  None  

[5 rows x 674 columns]

In [11]:
applabel = ['app_{0:03d}'.format(i) for i in list(set(merge.app))]
d.columns = applabel
d.head()

,app_000,app_001,app_002,app_003,app_004,app_005,app_006,app_007,app_008,app_009,...,app_758,app_759,app_760,app_761,app_763,app_764,app_765,app_766,app_767,app_768
ip_os_device_dow_hour,,,,,,,,,,,,,,,,,,,,,
10020001303,None,None,True,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10020001314,None,True,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10040001312,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10040001314,None,None,None,None,None,True,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10090001303,None,True,True,None,None,None,None,None,True,True,...,None,None,None,None,None,None,None,None,None,None


In [12]:
d.fillna(False, inplace=True)

In [13]:
print(d.shape)
d.head()

(14246745, 674)


,app_000,app_001,app_002,app_003,app_004,app_005,app_006,app_007,app_008,app_009,...,app_758,app_759,app_760,app_761,app_763,app_764,app_765,app_766,app_767,app_768
ip_os_device_dow_hour,,,,,,,,,,,,,,,,,,,,,
10020001303,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10020001314,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10040001312,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10040001314,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10090001303,False,True,True,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False


In [14]:
%%time
mat = np.dot(d.astype('bool').astype('uint16').values.T,
             d.astype('bool').astype('uint16').values)

CPU times: user 1h 27min 2s, sys: 3min 41s, total: 1h 30min 44s
Wall time: 1h 30min 43s


In [15]:
count_app_coincidence_per_user = pd.DataFrame(mat)

In [16]:
del mat, d
gc.collect()

64

In [17]:
count_app_coincidence_per_user.columns = applabel
count_app_coincidence_per_user.head()

,app_000,app_001,app_002,app_003,app_004,app_005,app_006,app_007,app_008,app_009,...,app_758,app_759,app_760,app_761,app_763,app_764,app_765,app_766,app_767,app_768
0,1417,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,54185,36169,28025,34987,27345,49964,25499,53798,19446,...,0,0,0,0,0,0,0,0,0,0
2,0,36169,45153,5804,24341,13147,18622,47144,55925,45289,...,0,0,0,0,0,0,0,1,0,0
3,0,28025,5804,10811,35971,26816,47986,53072,5199,25247,...,0,0,0,0,0,0,0,0,0,0
4,0,34987,24341,35971,38886,5188,8401,30969,6531,25491,...,0,0,0,0,0,0,0,0,0,0


In [18]:
count_app_coincidence_per_user.to_feather('../data/count_app_coincidence_per_user.feather')

In [19]:
del count_app_coincidence_per_user
gc.collect()

6078

In [21]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu 941K May  5 18:32 ../data/count_app_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 13:04 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### pca

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [13]:
d = pd.concat([
    pd.read_feather('../data/agg_by_app.feather'),
    pd.read_feather('../data/uqcount_by_app.feather'),
    pd.read_feather('../data/count_user_by_app_channel.feather'),
    pd.read_feather('../data/count_app_coincidence_per_user.feather')
], axis = 1)

In [14]:
print(d.shape)
d.head()

(674, 882)


,app,count_by_app,attrib_by_app,uqcount_channel_by_app,uqcount_ip_by_app,uqcount_device_by_app,uqcount_os_by_app,uqcount_user_by_app,app,ch_000,...,app_758,app_759,app_760,app_761,app_763,app_764,app_765,app_766,app_767,app_768
0,0,2609,791,1,1139,227,14,1417,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3795936,617,32,113943,6,209,2479017,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,14878211,3504,27,135317,6,351,5288033,2,0,...,0,0,0,0,0,0,0,1,0,0
3,3,23614633,6900,47,155393,7,280,7219771,3,0,...,0,0,0,0,0,0,0,0,0,0
4,4,89155,3,1,10502,6,104,38886,4,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y = d[['app', 'attrib_by_app']].iloc[:, 1:]
X = d.drop(['app', 'attrib_by_app'], axis=1)

In [16]:
n_components = 20
pca = PCA(n_components = n_components)
res = pca.fit_transform(X.values)

In [17]:
res.shape

(674, 20)

In [18]:
pca.explained_variance_ratio_.cumsum()

array([0.95978084, 0.97242835, 0.97908496, 0.98385926, 0.98759442,
       0.99037131, 0.99227542, 0.99384014, 0.99516721, 0.99617551,
       0.99698442, 0.99752732, 0.99799636, 0.99837639, 0.99871753,
       0.99902559, 0.99921386, 0.99938674, 0.99950128, 0.99958148])

In [19]:
col = ['app_pca_{0:02d}'.format(i) for i in [i + 1 for i in range(20)]]
res = pd.DataFrame(res, columns=col)
app_pca = pd.concat([y.app, res], axis=1)

In [20]:
app_pca.to_feather('../data/app_pca.feather')

In [21]:
app_pca.head()

,app,app_pca_01,app_pca_02,app_pca_03,app_pca_04,app_pca_05,app_pca_06,app_pca_07,app_pca_08,app_pca_09,...,app_pca_11,app_pca_12,app_pca_13,app_pca_14,app_pca_15,app_pca_16,app_pca_17,app_pca_18,app_pca_19,app_pca_20
0,0,-2.037682e+05,1.246065e+04,-1369.293267,-1.795007e+03,8344.962894,3762.766205,3187.802645,745.671129,1437.931321,...,-792.716660,-1304.447752,-1043.904305,-1067.104339,-391.386678,-1796.016227,-2523.585279,751.949688,1160.087824,601.651171
1,1,4.221561e+06,-7.309511e+05,170138.127578,2.860806e+04,-753266.574941,-394047.623635,-172146.805833,-71969.993347,-569225.806399,...,-31178.530527,20686.157571,250990.748505,-128335.105668,-230860.164994,-41352.881656,-210289.447486,-297724.266649,66519.633866,4914.493686
2,2,1.558698e+07,-6.994194e+05,-700663.885828,2.253609e+06,973713.862039,-66838.518498,257238.505435,52919.519325,-241148.812124,...,27709.056328,29358.486584,-30725.136260,-17213.489596,-45889.441826,-3219.807304,15579.590681,-8377.647526,-3636.416699,2587.004772
3,3,2.470941e+07,3.638925e+06,569143.210683,-1.595341e+05,-175391.594135,-243415.491391,-138906.204099,-91154.177186,18499.166517,...,3161.990339,-1368.264863,1818.902070,-9889.253608,5544.962216,-3005.579530,-551.871614,-652.252825,2119.884840,1191.302651
4,4,-1.089792e+05,1.937964e+03,-1086.757269,1.378737e+03,-4283.936616,-1138.571653,-3808.769966,-2039.042716,-4815.138024,...,3976.341873,15255.661060,16998.330362,51428.410375,1220.231554,-7658.841088,26179.202867,-6897.868517,-17628.241187,-17030.769812


### とりあえず回帰してみる

In [22]:
import statsmodels.api as sm

/home/ubuntu/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [23]:
X1 = sm.add_constant(
    app_pca.iloc[:, 1:] / app_pca.iloc[:, 1:].max().max()
    , prepend=False
)

In [24]:
X1.head()

,app_pca_01,app_pca_02,app_pca_03,app_pca_04,app_pca_05,app_pca_06,app_pca_07,app_pca_08,app_pca_09,app_pca_10,...,app_pca_12,app_pca_13,app_pca_14,app_pca_15,app_pca_16,app_pca_17,app_pca_18,app_pca_19,app_pca_20,const
0,-0.008247,0.000504,-0.000055,-0.000073,0.000338,0.000152,0.000129,0.000030,0.000058,-0.000089,...,-0.000053,-0.000042,-0.000043,-0.000016,-0.000073,-0.000102,0.000030,0.000047,0.000024,1.0
1,0.170848,-0.029582,0.006886,0.001158,-0.030485,-0.015947,-0.006967,-0.002913,-0.023037,-0.006884,...,0.000837,0.010158,-0.005194,-0.009343,-0.001674,-0.008511,-0.012049,0.002692,0.000199,1.0
2,0.630811,-0.028306,-0.028356,0.091205,0.039407,-0.002705,0.010411,0.002142,-0.009759,0.002688,...,0.001188,-0.001243,-0.000697,-0.001857,-0.000130,0.000631,-0.000339,-0.000147,0.000105,1.0
3,1.000000,0.147269,0.023033,-0.006456,-0.007098,-0.009851,-0.005622,-0.003689,0.000749,0.000070,...,-0.000055,0.000074,-0.000400,0.000224,-0.000122,-0.000022,-0.000026,0.000086,0.000048,1.0
4,-0.004410,0.000078,-0.000044,0.000056,-0.000173,-0.000046,-0.000154,-0.000083,-0.000195,0.000396,...,0.000617,0.000688,0.002081,0.000049,-0.000310,0.001059,-0.000279,-0.000713,-0.000689,1.0


In [25]:
model = sm.GLM(y.attrib_by_app.values,
               X1.values,
               family=sm.families.Poisson(),
               offset = np.log(X.count_by_app.values)
              )

In [26]:
print(y.attrib_by_app.shape)
print(X1.shape)
print(X.count_by_app.shape)

(674,)
(674, 21)
(674,)


In [27]:
print(model.fit().summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  674
Model:                            GLM   Df Residuals:                      653
Model Family:                 Poisson   Df Model:                           20
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -2.7854e+05
Date:                Sat, 05 May 2018   Deviance:                   5.5588e+05
Time:                        18:43:10   Pearson chi2:                 4.92e+08
No. Iterations:                    13                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1           -11.4369      0.012   -923.096      0.000     -11.461     -11.413
x2            55.2019      0.109    507.508      0.0